In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import math

height = 256
width = 256
noise_type="mul"
std_deviation = 0.3

In [ ]:
def random_number(min, max):
  return random.randint(math.floor(min), math.floor(max))

In [ ]:
def create_triangle(rand_height_start, rand_height_end, rand_width_start, rand_width_end, image):
  ppt = np.array([
    [random_number(rand_width_start, rand_width_end), random_number(rand_height_start, rand_height_end)], 
    [random_number(rand_width_start, rand_width_end), random_number(rand_height_start, rand_height_end)], 
    [random_number(rand_width_start, rand_width_end), random_number(rand_height_start, rand_height_end)], 
  ], np.int32)

  ppt = ppt.reshape((-1, 1, 2))
  cv2.fillPoly(image, [ppt], (random_number(0, 255), random_number(0, 255), random_number(0, 255)), 8)

In [ ]:
def create_rectangle(rand_height_start, rand_height_end, rand_width_start, rand_width_end, image):
  ppt = np.array([
    [random_number(rand_width_start, rand_width_end), random_number(rand_height_start, rand_height_end)], 
    [random_number(rand_width_start, rand_width_end), random_number(rand_height_start, rand_height_end)], 
    [random_number(rand_width_start, rand_width_end), random_number(rand_height_start, rand_height_end)], 
    [random_number(rand_width_start, rand_width_end), random_number(rand_height_start, rand_height_end)], 
  ], np.int32)

  ppt = ppt.reshape((-1, 1, 2))
  cv2.fillPoly(image, [ppt], (random_number(0, 255), random_number(0, 255), random_number(0, 255)), 8)

In [ ]:
def create_ellipse(height, width, image):
  cv2.ellipse(
    image,
    (random_number(0, width), random_number(0, height)),
    (random_number(0, width), random_number(0, height)),
    random_number(0, 360),
    0,
    360,
    (random_number(0, 255), random_number(0, 255), random_number(0, 255)),
    -1,
    8
  )

In [ ]:
def create_stars(height, width, image):
  points = []
  for i in range(5):
    points.append((random_number(0, width), random_number(0, height)))

  center = np.array(points).mean(axis = 0)

  for point in points:
    cv2.line(
      image,
      point,
      (math.floor(center[0]), math.floor(center[1])),
      (random_number(0, 255), random_number(0, 255), random_number(0, 255)),
      2,
      8
    )


In [ ]:
def generate_image(height, width, triangles, rectangles, ellipses, stars):
  r = np.full((height, width), random_number(0, 255))
  g = np.full((height, width), random_number(0, 255))
  b = np.full((height, width), random_number(0, 255))

  rand_height_start = height * -0.1
  rand_height_end = height * 1.1 
  rand_width_start = width * -0.1
  rand_width_end = width * 1.1 

  image = np.dstack((r, g, b))
  shapes = [triangles, rectangles, ellipses, stars]
  index = random_number(0, 3)
  print(sum(shapes))
  while sum(shapes) > 0:
    while(shapes[index] == 0):
      index = random_number(0, 3)
    
    if(index == 0):
      create_triangle(rand_height_start, rand_height_end, rand_width_start, rand_width_end, image)
    elif(index == 1):
      create_rectangle(rand_height_start, rand_height_end, rand_width_start, rand_width_end, image)
    elif(index == 2):
      create_ellipse(height, width, image)
    elif(index == 3):
      create_stars(height, width, image)

    shapes[index] -= 1

  return image

In [ ]:
def add_noise(image, type):
  if type == "add":
    noise = np.random.normal(0, std_deviation, image.shape)
    image = image + noise
  elif type == "mul":
    noise = np.random.normal(1, std_deviation, image.shape)
    image = image * noise

  return image


In [ ]:
image = generate_image(height, width, 3, 3, 3, 3)
image = image / 255

imgplot = plt.imshow(image)
plt.axis('off')
plt.show()

In [ ]:
noised_image = add_noise(image, noise_type)
noised_image[noised_image > 1] = 1

imgplot = plt.imshow(noised_image)
plt.axis('off')
plt.show()